In [ ]:
pd.set_option('display.max_colwidth',1000)


In [ ]:
import pandas as pd

url = "https://raw.githubusercontent.com/lintool/cs-big-cows/master/acm-fellows/acm_fellows.csv"
df = pd.read_csv(url)

In [ ]:
df.head()

,Index,Last Name,Given Name,Year,Location,Citation,ACM Fellow Profile,DBLP profile,Google Scholar Profile,Affiliation,Interests
0,1,Hassan,Ahmed E.,2022,Canada,For contributions to the quality assurance of ...,https://awards.acm.org/award-recipients/hassan...,https://dblp.org/pid/161/8687,https://scholar.google.com/citations?user=9hwX...,"ACM Fellow, IEEE Fellow, NSERC Steacie Fellow,...",Mining Software Repositories Software Analytic...
1,2,Pothen,Alex,2022,USA,For contributions to and leadership in combina...,https://awards.acm.org/award-recipients/pothen...,https://dblp.org/pid/03/854,https://scholar.google.com/citations?user=_Fah...,"Professor of Computer Science, Purdue University",Combinatorial Scientific Computing Parallel Co...
2,3,Kemper,Alfons,2022,Germany,For contributions to database management syste...,https://awards.acm.org/award-recipients/kemper...,https://dblp.org/pid/k/AlfonsKemper,NaN,NaN,[]
3,4,Anandkumar,Anima,2022,USA,For contributions to tensor methods for probab...,https://awards.acm.org/award-recipients/anandk...,https://dblp.org/pid/25/1529,https://scholar.google.com/citations?user=bEcL...,California Institute of Technology and NVIDIA,Machine Learning and Artificial Intelligence
4,5,Sabharwal,Ashutosh,2022,USA,For the invention of full-duplex wireless and ...,https://awards.acm.org/award-recipients/sabhar...,https://dblp.org/pid/45/284,https://scholar.google.com/citations?user=MiqM...,Department Chair and Ernest D. Butcher Profess...,Wireless Communications Wireless Platforms Dig...


In [ ]:
df['Citation'] = df['Citation'].str.strip()

In [ ]:
df['Citation'].isna().sum()

25

In [ ]:
df.dropna(subset=['Citation'], inplace=True)

In [ ]:
df[~df['Citation'].str.startswith('For contributions to', na=False)]['Citation']

4       For the invention of full-duplex wireless and ...
9       For fundamental contributions to algorithmic g...
10      For theoretical and practical contributions to...
14      For contributions in speech processing and dee...
21      For theoretical and practical contributions to...
                              ...                        
1434    For their vision and leadership in the design,...
1435    For dedicated, tireless and unflagging leaders...
1437    For outstanding contributions over an extended...
1438    For his basic research on parallel algorithms ...
1440    For his fundamental contributions to numerical...
Name: Citation, Length: 488, dtype: object

In [ ]:
!pip install nltk altair umap-learn cohere

In [ ]:
import string

import altair as alt
import cohere
import nltk
import numpy as np
import pandas as pd
import umap
from nltk.corpus import stopwords
from sklearn.cluster import KMeans
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.manifold import TSNE

In [ ]:
nltk.download('stopwords')

df['Citation'] = df['Citation'].str.lower()

custom_words_to_remove = ['contributions', 'leadership', 'pioneering', 'outstanding', 'seminal', 'fundamental', 'research']

def remove_stop_and_custom_words(text):
    stop_words = set(stopwords.words("english"))
    words = text.split()
    filtered_words = [word for word in words if word.lower() not in stop_words and word.lower() not in custom_words_to_remove]
    filtered_words = [''.join(char for char in word if char not in string.punctuation) for word in filtered_words]
    return " ".join(filtered_words)

df['citation_sw'] = df['Citation'].apply(remove_stop_and_custom_words)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
df['citation_sw']

0          quality assurance large scale software systems
1                      combinatorial scientific computing
2                   database management system technology
3       tensor methods probabilistic models neural ope...
4       invention fullduplex wireless opensource wirel...
                              ...                        
1434    vision design implementation evolution dissemi...
1435    dedicated tireless unflagging service acm sist...
1437    extended period acm scholastic programming con...
1438    basic parallel algorithms conception design im...
1440    numerical analysis one foremost experts floati...
Name: citation_sw, Length: 1419, dtype: object

In [ ]:
word_counts = df['citation_sw'].str.split().explode().value_counts()
word_counts_df = pd.DataFrame({'Word': word_counts.index, 'Count': word_counts.values})
word_counts_df[:10]

,Word,Count
0,systems,320
1,computer,316
2,computing,252
3,design,201
4,data,169
5,theory,164
6,software,150
7,algorithms,144
8,science,132
9,analysis,123


In [ ]:
api_key = 'eIH8kEo4m7R5YF03ZYkltqyBItxjtFnsT0W3okcC'
co = cohere.Client(api_key)

In [ ]:
model = 'embed-english-v2.0'
texts = df['citation_sw'].tolist()

df['embeds'] = co.embed(model=model,texts=texts)

In [ ]:
df.head()

,Index,Last Name,Given Name,Year,Location,Citation,ACM Fellow Profile,DBLP profile,Google Scholar Profile,Affiliation,Interests,citation_sw,embeds
0,1,Hassan,Ahmed E.,2022,Canada,for contributions to the quality assurance of ...,https://awards.acm.org/award-recipients/hassan...,https://dblp.org/pid/161/8687,https://scholar.google.com/citations?user=9hwX...,"ACM Fellow, IEEE Fellow, NSERC Steacie Fellow,...",Mining Software Repositories Software Analytic...,quality assurance large scale software systems,"[-1.6806641, 1.6484375, -1.7626953, -2.3261719..."
1,2,Pothen,Alex,2022,USA,for contributions to and leadership in combina...,https://awards.acm.org/award-recipients/pothen...,https://dblp.org/pid/03/854,https://scholar.google.com/citations?user=_Fah...,"Professor of Computer Science, Purdue University",Combinatorial Scientific Computing Parallel Co...,combinatorial scientific computing,"[1.7001953, 1.546875, 0.390625, -2.1445312, 1...."
2,3,Kemper,Alfons,2022,Germany,for contributions to database management syste...,https://awards.acm.org/award-recipients/kemper...,https://dblp.org/pid/k/AlfonsKemper,NaN,NaN,[],database management system technology,"[2.4648438, 0.53808594, 1.3134766, -2.2089844,..."
3,4,Anandkumar,Anima,2022,USA,for contributions to tensor methods for probab...,https://awards.acm.org/award-recipients/anandk...,https://dblp.org/pid/25/1529,https://scholar.google.com/citations?user=bEcL...,California Institute of Technology and NVIDIA,Machine Learning and Artificial Intelligence,tensor methods probabilistic models neural ope...,"[1.3154297, 0.5761719, -2.2695312, -0.2376709,..."
4,5,Sabharwal,Ashutosh,2022,USA,for the invention of full-duplex wireless and ...,https://awards.acm.org/award-recipients/sabhar...,https://dblp.org/pid/45/284,https://scholar.google.com/citations?user=MiqM...,Department Chair and Ernest D. Butcher Profess...,Wireless Communications Wireless Platforms Dig...,invention fullduplex wireless opensource wirel...,"[1.1357422, 0.3317871, 1.8554688, -0.85498047,..."


In [ ]:
embeddings = np.array(df['embeds'].tolist())
embeddings.shape

(1419, 4096)

In [ ]:
df['Full Name'] = df['Given Name'] + ' ' + df['Last Name']

In [ ]:
reducer = TSNE(n_components=2, perplexity=30, n_iter=300, random_state=42)

tsne_embeds = reducer.fit_transform(embeddings)

df_tsne_vis = pd.DataFrame(data={'Citation': df['Citation']})
df_tsne_vis['x'] = tsne_embeds[:, 0]
df_tsne_vis['y'] = tsne_embeds[:, 1]

reducer = umap.UMAP(n_neighbors=20, random_state=42)
umap_embeds = reducer.fit_transform(embeddings)

df_umap_vis = pd.DataFrame(data={'Citation': df['Citation']})
df_umap_vis['x'] = umap_embeds[:,0]
df_umap_vis['y'] = umap_embeds[:,1]

/usr/local/lib/python3.10/dist-packages/umap/umap_.py:1943: UserWarning: n_jobs value -1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(f"n_jobs value {self.n_jobs} overridden to 1 by setting random_state. Use no seed for parallelism.")


In [ ]:
df_tsne_vis.head()

,Citation,x,y
0,for contributions to the quality assurance of ...,-2.450605,-3.616478
1,for contributions to and leadership in combina...,5.481236,0.708201
2,for contributions to database management syste...,-0.236052,5.299500
3,for contributions to tensor methods for probab...,4.969205,3.489367
4,for the invention of full-duplex wireless and ...,-10.383647,0.616950


## T-SNE visualization

In [ ]:
chart = alt.Chart(df_tsne_vis).mark_circle(size=60).encode(
    x=alt.X('x', scale=alt.Scale(zero=False)),
    y=alt.Y('y', scale=alt.Scale(zero=False)),
    tooltip=['Citation']
).properties(
    width=1000,
    height=700
)
chart.interactive()

alt.Chart(...)

## Umap visualization

In [ ]:
chart = alt.Chart(df_umap_vis).mark_circle(size=60).encode(
    x=alt.X('x', scale=alt.Scale(zero=False)),
    y=alt.Y('y', scale=alt.Scale(zero=False)),
    tooltip=['Citation']
).properties(
    width=1000,
    height=700
)
chart.interactive()

alt.Chart(...)

## Cluster data with K-Means algorithm

In [ ]:
n_clusters = 10

kmeans_model = KMeans(n_clusters=n_clusters, random_state=42)
classes = kmeans_model.fit_predict(embeddings)

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


In [ ]:
df.head()

,Index,Last Name,Given Name,Year,Location,Citation,ACM Fellow Profile,DBLP profile,Google Scholar Profile,Affiliation,Interests,citation_sw,embeds,Full Name
0,1,Hassan,Ahmed E.,2022,Canada,for contributions to the quality assurance of ...,https://awards.acm.org/award-recipients/hassan...,https://dblp.org/pid/161/8687,https://scholar.google.com/citations?user=9hwX...,"ACM Fellow, IEEE Fellow, NSERC Steacie Fellow,...",Mining Software Repositories Software Analytic...,quality assurance large scale software systems,"[-1.6806641, 1.6484375, -1.7626953, -2.3261719...",Ahmed E. Hassan
1,2,Pothen,Alex,2022,USA,for contributions to and leadership in combina...,https://awards.acm.org/award-recipients/pothen...,https://dblp.org/pid/03/854,https://scholar.google.com/citations?user=_Fah...,"Professor of Computer Science, Purdue University",Combinatorial Scientific Computing Parallel Co...,combinatorial scientific computing,"[1.7001953, 1.546875, 0.390625, -2.1445312, 1....",Alex Pothen
2,3,Kemper,Alfons,2022,Germany,for contributions to database management syste...,https://awards.acm.org/award-recipients/kemper...,https://dblp.org/pid/k/AlfonsKemper,NaN,NaN,[],database management system technology,"[2.4648438, 0.53808594, 1.3134766, -2.2089844,...",Alfons Kemper
3,4,Anandkumar,Anima,2022,USA,for contributions to tensor methods for probab...,https://awards.acm.org/award-recipients/anandk...,https://dblp.org/pid/25/1529,https://scholar.google.com/citations?user=bEcL...,California Institute of Technology and NVIDIA,Machine Learning and Artificial Intelligence,tensor methods probabilistic models neural ope...,"[1.3154297, 0.5761719, -2.2695312, -0.2376709,...",Anima Anandkumar
4,5,Sabharwal,Ashutosh,2022,USA,for the invention of full-duplex wireless and ...,https://awards.acm.org/award-recipients/sabhar...,https://dblp.org/pid/45/284,https://scholar.google.com/citations?user=MiqM...,Department Chair and Ernest D. Butcher Profess...,Wireless Communications Wireless Platforms Dig...,invention fullduplex wireless opensource wirel...,"[1.1357422, 0.3317871, 1.8554688, -0.85498047,...",Ashutosh Sabharwal


In [ ]:
documents = df['Citation']
documents = pd.DataFrame({"Document": documents,
                          "ID": range(len(documents)),
                          "Topic": None})
documents['Topic'] = classes
documents_per_topic = documents.groupby(['Topic'], as_index=False).agg({'Document': ' '.join})

# Use scikit-learn's TfidfVectorizer to compute TF-IDF values
tfidf_vectorizer = TfidfVectorizer(stop_words="english").fit(documents_per_topic.Document)
tfidf = tfidf_vectorizer.transform(documents_per_topic.Document)
tfidf_matrix = tfidf.toarray()

words = tfidf_vectorizer.get_feature_names_out()
words_per_class = {label: [words[index] for index in tfidf_matrix[i].argsort()[-6:]] for i, label in enumerate(documents_per_topic.Topic)}
df['cluster'] = classes
df['keywords'] = df['cluster'].map(lambda topic_num: ", ".join(np.array(words_per_class[topic_num])[:]))

In [ ]:
words_per_class = {key: [word for word in value if word != 'contributions'] for key, value in words_per_class.items()}
words_per_class

{0: ['cryptographic', 'systems', 'cryptography', 'privacy', 'security'],
 1: ['machine', 'natural', 'artificial', 'intelligence', 'learning'],
 2: ['combinatorial', 'computational', 'theory', 'complexity', 'algorithms'],
 3: ['systems', 'verification', 'languages', 'software', 'programming'],
 4: ['service', 'science', 'computing', 'acm', 'computer'],
 5: ['query', 'systems', 'data', 'management', 'database'],
 6: ['rendering', 'image', 'graphics', 'computer', 'vision'],
 7: ['cooperative', 'computing', 'computer', 'interaction', 'human'],
 8: ['protocols', 'internet', 'systems', 'networks', 'wireless'],
 9: ['memory', 'systems', 'distributed', 'parallel', 'computing']}

In [ ]:
from urllib.parse import urlparse, parse_qs, urljoin
import numpy as np

def find_image_url(scholar_url):
    if not (scholar_url and isinstance(scholar_url, str)):
        return np.nan
    if not scholar_url.startswith("https://scholar.google"):
        return np.nan

    parsed_url = urlparse(scholar_url)
    query_parameters = parse_qs(parsed_url.query)
    user_id = query_parameters.get("user", [None])[0]

    if user_id is None:
        return np.nan

    image_url = urljoin("https://scholar.googleusercontent.com/", f"citations?view_op=medium_photo&user={user_id}")
    return image_url

df['image_url'] = df['Google Scholar Profile'].apply(find_image_url)

In [ ]:
col_to_move = ['Index', 'Full Name', 'ACM Fellow Profile', 'DBLP profile', 'Google Scholar Profile', 'image_url']

df_tsne_vis['cluster'] = classes
df_tsne_vis['keywords'] = df['cluster'].map(lambda topic_num: ", ".join(np.array(words_per_class[topic_num])[:]))
df_tsne_vis[col_to_move] = df[col_to_move]

df_umap_vis['cluster'] = classes
df_umap_vis['keywords'] = df['cluster'].map(lambda topic_num: ", ".join(np.array(words_per_class[topic_num])[:]))
df_umap_vis[col_to_move] = df[col_to_move]

In [ ]:
df_tsne_vis

,Citation,x,y,cluster,keywords,Index,Full Name,ACM Fellow Profile,DBLP profile,Google Scholar Profile,image_url
0,for contributions to the quality assurance of ...,-2.450605,-3.616478,3,"systems, verification, languages, software, pr...",1,Ahmed E. Hassan,https://awards.acm.org/award-recipients/hassan...,https://dblp.org/pid/161/8687,https://scholar.google.com/citations?user=9hwX...,https://scholar.googleusercontent.com/citation...
1,for contributions to and leadership in combina...,5.481236,0.708201,2,"combinatorial, computational, theory, complexi...",2,Alex Pothen,https://awards.acm.org/award-recipients/pothen...,https://dblp.org/pid/03/854,https://scholar.google.com/citations?user=_Fah...,https://scholar.googleusercontent.com/citation...
2,for contributions to database management syste...,-0.236052,5.299500,5,"query, systems, data, management, database",3,Alfons Kemper,https://awards.acm.org/award-recipients/kemper...,https://dblp.org/pid/k/AlfonsKemper,NaN,NaN
3,for contributions to tensor methods for probab...,4.969205,3.489367,1,"machine, natural, artificial, intelligence, le...",4,Anima Anandkumar,https://awards.acm.org/award-recipients/anandk...,https://dblp.org/pid/25/1529,https://scholar.google.com/citations?user=bEcL...,https://scholar.googleusercontent.com/citation...
4,for the invention of full-duplex wireless and ...,-10.383647,0.616950,8,"protocols, internet, systems, networks, wireless",5,Ashutosh Sabharwal,https://awards.acm.org/award-recipients/sabhar...,https://dblp.org/pid/45/284,https://scholar.google.com/citations?user=MiqM...,https://scholar.googleusercontent.com/citation...
...,...,...,...,...,...,...,...,...,...,...,...
1434,"for their vision and leadership in the design,...",-10.099010,-2.011267,8,"protocols, internet, systems, networks, wireless",1435,Vinton Cerf,https://awards.acm.org/award-recipients/cerf_1...,https://dblp.org/pid/c/VintonGCerf,NaN,NaN
1435,"for dedicated, tireless and unflagging leaders...",0.596543,-1.017723,4,"service, science, computing, acm, computer",1436,Walter Carlson,https://awards.acm.org/award-recipients/carlso...,https://dblp.org/pid/75/5922,https://scholar.google.com/citations?user=IEcm...,https://scholar.googleusercontent.com/citation...
1437,for outstanding contributions over an extended...,2.313103,-4.000226,4,"service, science, computing, acm, computer",1438,William B Poucher,https://awards.acm.org/award-recipients/pouche...,https://dblp.org/pid/20/10566,NaN,NaN
1438,for his basic research on parallel algorithms ...,-4.426122,-1.039151,9,"memory, systems, distributed, parallel, computing",1439,William Daniel Hillis,https://awards.acm.org/award-recipients/hillis...,NaN,NaN,NaN


In [ ]:
df_umap_vis['Google Scholar Profile'].isna().sum()

481

In [ ]:
df_umap_vis['image_url'].isna().sum()

482

In [ ]:
df_umap_vis['image_url'].notna().sum()

937

In [ ]:
df.head()

,Index,Last Name,Given Name,Year,Location,Citation,ACM Fellow Profile,DBLP profile,Google Scholar Profile,Affiliation,Interests,citation_sw,embeds,Full Name,cluster,keywords,image_url
0,1,Hassan,Ahmed E.,2022,Canada,for contributions to the quality assurance of ...,https://awards.acm.org/award-recipients/hassan...,https://dblp.org/pid/161/8687,https://scholar.google.com/citations?user=9hwX...,"ACM Fellow, IEEE Fellow, NSERC Steacie Fellow,...",Mining Software Repositories Software Analytic...,quality assurance large scale software systems,"[-1.6806641, 1.6484375, -1.7626953, -2.3261719...",Ahmed E. Hassan,3,"systems, verification, languages, software, pr...",https://scholar.googleusercontent.com/citation...
1,2,Pothen,Alex,2022,USA,for contributions to and leadership in combina...,https://awards.acm.org/award-recipients/pothen...,https://dblp.org/pid/03/854,https://scholar.google.com/citations?user=_Fah...,"Professor of Computer Science, Purdue University",Combinatorial Scientific Computing Parallel Co...,combinatorial scientific computing,"[1.7001953, 1.546875, 0.390625, -2.1445312, 1....",Alex Pothen,2,"combinatorial, computational, theory, complexi...",https://scholar.googleusercontent.com/citation...
2,3,Kemper,Alfons,2022,Germany,for contributions to database management syste...,https://awards.acm.org/award-recipients/kemper...,https://dblp.org/pid/k/AlfonsKemper,NaN,NaN,[],database management system technology,"[2.4648438, 0.53808594, 1.3134766, -2.2089844,...",Alfons Kemper,5,"query, systems, data, management, database, co...",NaN
3,4,Anandkumar,Anima,2022,USA,for contributions to tensor methods for probab...,https://awards.acm.org/award-recipients/anandk...,https://dblp.org/pid/25/1529,https://scholar.google.com/citations?user=bEcL...,California Institute of Technology and NVIDIA,Machine Learning and Artificial Intelligence,tensor methods probabilistic models neural ope...,"[1.3154297, 0.5761719, -2.2695312, -0.2376709,...",Anima Anandkumar,1,"machine, natural, artificial, intelligence, le...",https://scholar.googleusercontent.com/citation...
4,5,Sabharwal,Ashutosh,2022,USA,for the invention of full-duplex wireless and ...,https://awards.acm.org/award-recipients/sabhar...,https://dblp.org/pid/45/284,https://scholar.google.com/citations?user=MiqM...,Department Chair and Ernest D. Butcher Profess...,Wireless Communications Wireless Platforms Dig...,invention fullduplex wireless opensource wirel...,"[1.1357422, 0.3317871, 1.8554688, -0.85498047,...",Ashutosh Sabharwal,8,"protocols, internet, systems, networks, wirele...",https://scholar.googleusercontent.com/citation...


In [ ]:
df.head()[['Citation', 'keywords']]

,Citation,keywords
0,for contributions to the quality assurance of ...,"systems, verification, languages, software, pr..."
1,for contributions to and leadership in combina...,"combinatorial, computational, theory, complexi..."
2,for contributions to database management syste...,"query, systems, data, management, database, co..."
3,for contributions to tensor methods for probab...,"machine, natural, artificial, intelligence, le..."
4,for the invention of full-duplex wireless and ...,"protocols, internet, systems, networks, wirele..."


In [ ]:
df_umap_vis.head()

,Citation,x,y,cluster,keywords,Index,Full Name,ACM Fellow Profile,DBLP profile,Google Scholar Profile
0,for contributions to the quality assurance of ...,-2.388297,7.772640,3,"systems, verification, languages, software, pr...",1,Ahmed E. Hassan,https://awards.acm.org/award-recipients/hassan...,https://dblp.org/pid/161/8687,https://scholar.google.com/citations?user=9hwX...
1,for contributions to and leadership in combina...,2.852165,8.820793,2,"combinatorial, computational, theory, complexi...",2,Alex Pothen,https://awards.acm.org/award-recipients/pothen...,https://dblp.org/pid/03/854,https://scholar.google.com/citations?user=_Fah...
2,for contributions to database management syste...,1.945701,6.161602,5,"query, systems, data, management, database",3,Alfons Kemper,https://awards.acm.org/award-recipients/kemper...,https://dblp.org/pid/k/AlfonsKemper,NaN
3,for contributions to tensor methods for probab...,3.689605,7.288111,1,"machine, natural, artificial, intelligence, le...",4,Anima Anandkumar,https://awards.acm.org/award-recipients/anandk...,https://dblp.org/pid/25/1529,https://scholar.google.com/citations?user=bEcL...
4,for the invention of full-duplex wireless and ...,2.753146,3.542735,8,"protocols, internet, systems, networks, wireless",5,Ashutosh Sabharwal,https://awards.acm.org/award-recipients/sabhar...,https://dblp.org/pid/45/284,https://scholar.google.com/citations?user=MiqM...


## T-SNE Visualization with K-Means Clustering

In [ ]:
selection = alt.selection_multi(fields=['keywords'], bind='legend')

chart = alt.Chart(df_tsne_vis).mark_circle(size=60, stroke='#666', strokeWidth=1, opacity=0.3).encode(
    x=alt.X('x',
            scale=alt.Scale(zero=False),
            axis=alt.Axis(labels=False, ticks=False, domain=False)),
    y=alt.Y('y',
            scale=alt.Scale(zero=False),
            axis=alt.Axis(labels=False, ticks=False, domain=False)),
    color=alt.Color('keywords:N',
                    legend=alt.Legend(columns=1, labelFontSize=14)),
    opacity=alt.condition(selection, alt.value(1), alt.value(0.2)),
    tooltip=['Citation', 'keywords', 'Full Name', 'Index', 'cluster']
).properties(
    width=1000,
    height=700
).add_selection(
    selection
).configure_legend(labelLimit=0).configure_view(
    strokeWidth=0
).configure(background="#FDF7F0").properties(
    title='T-SNE Visualization'
)

chart.interactive()

alt.Chart(...)

## UMAP Visualization with K-Means Clustering

In [ ]:
selection = alt.selection_multi(fields=['keywords'], bind='legend')

chart = alt.Chart(df_umap_vis).mark_circle(size=60, stroke='#666', strokeWidth=1, opacity=0.3).encode(
    x=alt.X('x',
            scale=alt.Scale(zero=False),
            axis=alt.Axis(labels=False, ticks=False, domain=False)),
    y=alt.Y('y',
            scale=alt.Scale(zero=False),
            axis=alt.Axis(labels=False, ticks=False, domain=False)),
    color=alt.Color('keywords:N',
                    legend=alt.Legend(columns=1, labelFontSize=14)),
    opacity=alt.condition(selection, alt.value(1), alt.value(0.2)),
    tooltip=['Citation', 'keywords', 'Full Name', 'Index', 'cluster']
).properties(
    width=1000,
    height=700
).add_selection(
    selection
).configure_legend(labelLimit=0).configure_view(
    strokeWidth=0
).configure(background="#FDF7F0").properties(
    title='UMAP Visualization'
)

chart.interactive()

alt.Chart(...)

In [ ]:
df_umap_vis.head()

,Citation,x,y,cluster,keywords,Index,Full Name,ACM Fellow Profile,DBLP profile,Google Scholar Profile
0,for contributions to the quality assurance of ...,-2.388297,7.772640,3,"systems, verification, languages, software, pr...",1,Ahmed E. Hassan,https://awards.acm.org/award-recipients/hassan...,https://dblp.org/pid/161/8687,https://scholar.google.com/citations?user=9hwX...
1,for contributions to and leadership in combina...,2.852165,8.820793,2,"combinatorial, computational, theory, complexi...",2,Alex Pothen,https://awards.acm.org/award-recipients/pothen...,https://dblp.org/pid/03/854,https://scholar.google.com/citations?user=_Fah...
2,for contributions to database management syste...,1.945701,6.161602,5,"query, systems, data, management, database",3,Alfons Kemper,https://awards.acm.org/award-recipients/kemper...,https://dblp.org/pid/k/AlfonsKemper,NaN
3,for contributions to tensor methods for probab...,3.689605,7.288111,1,"machine, natural, artificial, intelligence, le...",4,Anima Anandkumar,https://awards.acm.org/award-recipients/anandk...,https://dblp.org/pid/25/1529,https://scholar.google.com/citations?user=bEcL...
4,for the invention of full-duplex wireless and ...,2.753146,3.542735,8,"protocols, internet, systems, networks, wireless",5,Ashutosh Sabharwal,https://awards.acm.org/award-recipients/sabhar...,https://dblp.org/pid/45/284,https://scholar.google.com/citations?user=MiqM...


In [ ]:
df_tsne_vis.to_csv('tsne_visualization.csv', index=False)

In [ ]:
df_umap_vis.to_csv('umap_visualization.csv', index=False)